# Tugas-2 Server

## Web Server

In [ ]:
# import library socket
import socket as sc

# Definisikan IP address
IP = 'localhost'
# Port HTTP adalah 80
PORT = 80

# Definisikan socket dengan tipe TCP
s = sc.socket(sc.AF_INET, sc.SOCK_STREAM)

# Lakukan binding pada socket 
s.bind((IP, PORT))

# Socket listen pada IP dan Port yang sudah didefinisikan
s.listen()

# Tampilkan bahwa server sudah siap melayani request
print(f'Server sudah berjalan dan siap melayani pada: {IP}:{PORT}')

# Loop forever
while True:
    # Socket menerima request
    conn, addr = s.accept()
    
    # Socket menerima data
    RECEIVED = conn.recv(1024)
    
    # Print data hasil koneksi
    print(RECEIVED)
    
    # Buat respone HTTP untuk client
    http_response = """\HTTP/1.1 200 OK

<html>
<head>
<title>Web Server Sederhana</title>
</head>
<body>

<h1>Heading 1</h1>
<p>Ini adalah contoh paragraf.</p>
<img src="https://www.surfertoday.com/images/stories/surfetiquette.jpg">

</body>
</html>
"""
    # Kirim response kepada client dengan sendall() jangan lupa diencode response dengan utf-8 
    conn.sendall(http_response.encode('utf-8'))
    
    # Tutup koneksi
    conn.close()

# Selamat! Kamu telah berhasil membuat web server sederhana. 

## Sistem Voting

### Satu server

In [ ]:
# Import library SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCServer

# import library SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCRequestHandler

# import library untuk threading locking
import threading

# Definisikan IP address dan port
IP = 'localhost'
PORT = 8080

# Batasi path (folder) yang bisa diakses
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)
    
# Buat server
with SimpleXMLRPCServer((IP, PORT),
                        requestHandler=RequestHandler) as server:
    server.register_introspection_functions()

    # Data struktur dictionary kandidat voting
    candidate = {    
        "Burhan": 0,
        "Jamal": 0,
        "Reinhard": 0
    }
    
    # assign threading.Lock() karena ada fungsi yang critical section yang akan menyebabkan race condition
    lock = threading.Lock()
    
    # Fungsi untuk vote kandidat dengan input nama kandidatnya
    def vote_add(name: str) -> str:
        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        # jika kandidat ada dalam dictionary maka tambahkan nilai votenya
        if name in candidate:
            candidate[name] += 1
            pesan = f'Hatur nuhun, voting untuk {name} telah diterima dengan selamat sentosa'
        else:
            pesan = f'Kandidat dengan nama {name} tidak terdaftar'
        
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return pesan
    
    # Registrasi fungsi vote_add() ke server
    server.register_function(vote_add, 'vote')

    # Fungsi untuk menampilkan hasil vote dari semua kandidat
    def vote_result() -> str:
        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        # Hitung total vote
        total_vote = sum([x for x in candidate.values()])
        
        persentase = 'Hasil voting = '
        
        # Jika belum ada voting
        if total_vote == 0:
            return 'Belum ada voting yang dilakukan'
        
        # hitung hasil persentase masing-masing kandidat
        for i in candidate:
            persentase += i+ ' ' + str(candidate[i]/total_vote*100)+'%, '
            
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return persentase
        
    # Registrasi fungsi vote_result() ke server
    server.register_function(vote_result, 'querry')

    print ("Server no voting ga starto desuyooo....")
    # Server START
    server.serve_forever()

## Unduh File

In [ ]:
# import library
from xmlrpc.server import SimpleXMLRPCServer
import xmlrpc.client

# fungsi untuk mendownload file
def unduh_file():
    # buka file yang akan didownload oleh client
    with open("file_diunduh.txt",'rb') as file:
        # baca file
        file_diunduh = file.read()
        # return file dengan format Binary dari library xmlrp.client
        return xmlrpc.client.Binary(file_diunduh)

# definisikan IP dan PORT
IP = 'localhost'
PORT = 8000
server = SimpleXMLRPCServer((IP, PORT))

print(f'Server siap pada alamat {IP}:{PORT}')

# registrasikan fungsi unduh_file() ke server
server.register_function(unduh_file, 'unduh')

# server start
server.serve_forever()